In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///onto.db', echo = True)

In [7]:
conn = engine.connect()
conn.execute("SELECT * FROM ontologies").fetchall()


2019-05-29 17:35:42,500 INFO sqlalchemy.engine.base.Engine SELECT * FROM ontologies
2019-05-29 17:35:42,506 INFO sqlalchemy.engine.base.Engine ()


[(1, 'pizza', 0), (2, 'pizza', 0)]

In [13]:
from rdflib.namespace import OWL, RDF, RDFS

def get_new_relations(filepath):
    d = dict()
    f = open(filepath, 'r')
    relations = list()
    classes = list()
    subclasses = list()

    # Each line of the new relations file is an RDF triple, so it is a
    # triple of the subject, predicate, and object
    # Create an adjacency list graph from the triples
    for l in f.readlines():
        s, p, o = l.split()
        if o == str(OWL.Class):
            classes.append(s)
        elif (p == str(RDFS.subClassOf) and not is_blank(s) and not is_blank(o)):
            subclasses.append((s, o))
        else:
            if s in d:
                d[s].append((p, o))
            else:
                d[s] = [(p, o)]

    # From the graph, find all restricitons (blank nodes) and get the relevant
    # relation data from them
    for s in d:
        if not is_blank(s):
            for p, o in d[s]:
                if is_blank(o):
                    domain = s
                    rang = None
                    quant = None
                    prop = None
                    for p1, o1 in d[o]:
                        if p1 == str(OWL.onProperty):
                            prop = o1
                        elif p1 == str(OWL.someValuesFrom):
                            quant = p1
                            rang = o1
                    if quant == str(OWL.someValuesFrom):
                        relations.append((domain, prop, quant, rang))
    print(subclasses)
    return relations, classes, subclasses


def is_blank(node):
    if not '#' in node:
        return True
    else:
        return False

get_new_relations("data/new/pizza.txt")

[]


([('http://www.co-ode.org/ontologies/pizza/pizza.owl#AmericanHot',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping',
   'http://www.w3.org/2002/07/owl#someValuesFrom',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping'),
  ('http://www.co-ode.org/ontologies/pizza/pizza.owl#AmericanHot',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping',
   'http://www.w3.org/2002/07/owl#someValuesFrom',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#HotGreenPepperTopping'),
  ('http://www.co-ode.org/ontologies/pizza/pizza.owl#AmericanHot',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping',
   'http://www.w3.org/2002/07/owl#someValuesFrom',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping'),
  ('http://www.co-ode.org/ontologies/pizza/pizza.owl#AmericanHot',
   'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping',
   'http://www.w3.org/2002/07/owl#someValuesFrom',
   'http://www.co-ode.org/ontologies/

In [9]:
import rdflib
ls = [tuple([eval(elem) for elem in line.split(" ", 2)]) for line in open("data/new/pizza2.txt").read().split("\n")]



In [10]:
from rdflib.term import Literal, URIRef, BNode
nonEmptyElems = [elem for elem in ls if (not type(elem[0]) == BNode) and (not type(elem[2]) == BNode)]


In [11]:
nonEmptyElems

[(rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#SpinachTopping'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('CoberturaDeEspinafre', lang='pt')),
 (rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#Capricciosa'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
  rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#NamedPizza')),
 (rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#RealItalianPizza'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('PizzaItalianaReal', lang='pt')),
 (rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#GoatsCheeseTopping'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('CoberturaDeQueijoDeCabra', lang='pt')),
 (rdflib.term.URIRef('http://www.co-ode.org/ontologies/pizza/pizza.owl#DeepPanBase'),
  rdflib.term.